# Stored Procedures
## Case
For our business we'd like to see which products are out of stock. So we can make sure to order some extra in the next delivery run. 


## How does the data look like?
First of all we need to know what the rows in the tables `Product` and `Supplier` contain.

In [2]:
-- Product Table Example
SELECT TOP 3 * FROM Product
-- Supplier Table Example
SELECT TOP 3 * FROM Supplier

The query that could tackle this problem is the one below.

In [3]:
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE UnitsInStock < 250

Since this is logic we'd like to reuse we can use a stored procedure, a reusable piece of code.
> Do note that a function for this case is also possible, but what is the difference between a function and a stored procedure?

## Create the stored procedure
### Syntax
```sql
CREATE PROCEDURE <proc_name> [parameter declaration] 
AS
<sql_statements> 
```


In [46]:
CREATE PROCEDURE GetProductsOutOfStock
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE UnitsInStock < 250

## Execute the procedure

In [48]:
EXECUTE GetProductsOutOfStock
-- EXEC is an alias for EXECUTE

## Delete a stored procedure


In [44]:
DROP PROCEDURE GetProductsOutOfStock  

## Change a stored procedure
We only want to see which products are out of stock if the `UnitsInStock` is less than `100` instead of `250`

In [47]:
ALTER PROCEDURE GetProductsOutOfStock
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE UnitsInStock < 100

## Using input parameters
Since we only want to see products of a certain supplier, we'll add an `input` parameter

In [49]:
ALTER PROCEDURE GetProductsOutOfStock 
    @supplierId INT
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE   UnitsInStock < 100
AND     SupplierId = @supplierId

## Execute the procedure with an input variable


In [50]:
EXECUTE GetProductsOutOfStock 2

## Using output parameters
Since we also want to see the name of the supplier, we'll add an `output` parameter, `supplierName`

In [51]:
ALTER PROCEDURE GetProductsOutOfStock 
    @supplierId INT,
    @supplierName NVARCHAR(MAX) OUTPUT
AS
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE   UnitsInStock < 100
AND     SupplierId = @supplierId

SELECT @supplierName = SupplierName
FROM Supplier
WHERE SupplierId = @supplierId


## Execute the procedure with an input and output variable




In [52]:
DECLARE @supplierName NVARCHAR(max)
EXECUTE GetProductsOutOfStock 2, @supplierName OUTPUT
PRINT 'Products out of stock for supplier:' + @supplierName

## Case - Deleting customers 


In [ ]:
create procedure DeleteCustomer
    @custno int = NULL
as

if @custno IS NULL
begin
    print 'Please provide a customerid'
    return
End

if not exists (select NULL
from customer
where customerid = @custno)
begin
    print 'The customer doesn''t exist.'
    return
End

if exists (select NULL
from orders
where customerid = @custno)
begin
    print 'The customer already has orders and can''t be deleted.'
    return
End

delete from customer where customerid = @custno
print 'The customer has been succesfully deleted'


# Functions
Are basically the same as a `stored procedure` and `view` but there are some (subtle) differences:
- A function **cannot** mutate data or database objects, it can only retrieve data or make calculations based on data.
> By using a `Function` you're implicitly saying you're not mutating the database nor it's data it contains. 
- A `view` **cannot** accept input parameters, a `function` can.

## Case
For our business we'd like to see which products are out of stock. So we can make sure to order some extra in the next delivery run, we want to specify the minimum amount of stock and a given supplier. 

## Table Valued Function
A table-valued function is a user-defined function that returns data of a table type. The return type of a table-valued function is a table, therefore, you can use the table-valued function just like you would use a table.

In [2]:
CREATE FUNCTION udf_GetProductsOutOfStockForSupplier(@minStock INT, @supplierId INT) RETURNS TABLE AS
RETURN 
SELECT
    ProductID as ID,
    ProductName as Name,
    UnitsInStock as Stock
FROM Product
WHERE   UnitsInStock < @minStock
    AND SupplierId = @supplierId

## Executing a Table Valued Function
Notice that this is not possible with a SQL View.

In [8]:
SELECT *
FROM udf_GetProductsOutOfStockForSupplier(45,7)